In [1]:
!nvidia-smi

Sun Apr 14 23:10:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers

In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
import torch
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel, TextDataset, LineByLineTextDataset,
    DataCollatorForLanguageModeling, TrainingArguments, Trainer, get_cosine_schedule_with_warmup
)

import matplotlib.pyplot as plt
%matplotlib inline

import json
import numpy as np
import seaborn as sns
import json
import nltk


In [ ]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json

--2024-04-14 19:41:41--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.212.24, 52.217.129.216, 52.217.140.112, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.212.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘personachat_self_original.json’

personachat_self_or 100%[===================>] 200.13M  54.9MB/s    in 3.6s    

2024-04-14 19:41:45 (54.9 MB/s) - ‘personachat_self_original.json’ saved [209850483/209850483]



In [5]:
bos_token = '<start>'
eos_token = '<end>'

In [ ]:
with open('/content/personachat_self_original.json') as json_file:
    data = json.load(json_file)

In [ ]:
chats = []
for i in range(len(data['train'])):
    personality = " ".join(data['train'][i]['personality'])
    chat = data['train'][i]['utterances'][-1]['history']
    final_text = bos_token + personality + '<start_chat>'
    for j in range(len(chat)):
        if j % 2 == 0:
            final_text += '<Human>'
        else:
            final_text += '<AI>'
        final_text += chat[j]
    chats.append(final_text + eos_token)

In [ ]:
chats = " ".join(chats)

In [ ]:
train_data_path = 'chats.txt'
with open(train_data_path, 'w') as t:
    t.write(chats)

# Tokenezation


In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token=bos_token, eos_token=eos_token, pad_token='<|pad|>')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
block_size = 1024

In [ ]:
train_dataset = TextDataset(tokenizer=tokenizer, file_path='/content/chats.txt', block_size=block_size, overwrite_cache=True)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
len(train_dataset)

4283

test dataset

In [ ]:
# Process test data in a similar way
test_chats = []
for i in range(len(data['valid'])):
    personality = " ".join(data['valid'][i]['personality'])
    chat = data['valid'][i]['utterances'][-1]['history']
    final_text = bos_token + personality + '<start_chat>'
    for j in range(len(chat)):
        if j % 2 == 0:
            final_text += '<Human>'
        else:
            final_text += '<AI>'
        final_text += chat[j]
    test_chats.append(final_text + eos_token)
test_chats = " ".join(test_chats)

# Save processed test data to a text file
test_data_path = 'test_chats.txt'
with open(test_data_path, 'w') as t:
    t.write(test_chats)

# Create test dataset
test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_data_path, block_size=block_size, overwrite_cache=True)


# Training

In [ ]:
model = GPT2LMHeadModel.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model.cuda()
None

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

**inference with GPT 2 before training**

In [ ]:
def generate(prompt):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)  # Move model to the appropriate device

    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    eos_id = tokenizer.eos_token_id

    sample_outputs = model.generate(
        input_ids,
        max_length=len(input_ids[0]) + 50,
        min_length=len(input_ids[0]) + 30,
        do_sample=True,
        top_k=50,
        top_p=0.92,
        temperature=0.95,
        num_beams=1,
        eos_token_id=eos_id,
        num_return_sequences=1
    )

    for i, sample_output in enumerate(sample_outputs):
        sample_output = sample_output[len(input_ids[0]):]
        return tokenizer.decode(sample_output, skip_special_tokens=False)

In [ ]:
learning_rate = 2e-5
batch_size = 1
num_epochs = 2

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

seed = 42
training_args = TrainingArguments(
    output_dir='chats_output',
    overwrite_output_dir=True,
    do_train=True,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    prediction_loss_only=True,
    logging_steps=100,
    save_steps=300,
    save_total_limit=2,  # Keep only the last two checkpoints.
    evaluation_strategy="no",  # Set evaluation strategy to "no"
    save_strategy="no",  # Set save strategy to "no"
    load_best_model_at_end=True,
    seed=seed,
    learning_rate=learning_rate
)


In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader) * num_epochs
trainer.create_optimizer_and_scheduler(num_train_steps)

trainer.lr_scheduler = get_cosine_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=num_train_steps//10,
    num_training_steps=num_train_steps
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
100,2.181100
200,2.225100
300,2.240500
400,2.266100
500,2.300400
600,2.269400
700,2.264900
800,2.308400
900,2.262700
1000,2.258100


TrainOutput(global_step=8566, training_loss=2.1909914499906744, metrics={'train_runtime': 3101.7642, 'train_samples_per_second': 2.762, 'train_steps_per_second': 2.762, 'total_flos': 4476455092224000.0, 'train_loss': 2.1909914499906744, 'epoch': 2.0})

In [ ]:
eval_result = trainer.evaluate(eval_dataset=test_dataset)


In [ ]:
perplexity = torch.exp(torch.tensor(eval_result["eval_loss"])).item()

In [ ]:
print(perplexity)

11.477643013000488


# Save model

In [ ]:
output_dir = '/content/chats_output1'

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/chats_output1/tokenizer_config.json',
 '/content/chats_output1/special_tokens_map.json',
 '/content/chats_output1/vocab.json',
 '/content/chats_output1/merges.txt',
 '/content/chats_output1/added_tokens.json')

# Text generation


In [ ]:
def generate(prompt):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)  # Move model to the appropriate device

    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    eos_id = tokenizer.eos_token_id

    sample_outputs = model.generate(
        input_ids,
        max_length=len(input_ids[0]) + 50,
        min_length=len(input_ids[0]) + 30,
        do_sample=True,
        top_k=50,
        top_p=0.92,
        temperature=0.95,
        num_beams=1,
        eos_token_id=eos_id,
        num_return_sequences=1
    )

    for i, sample_output in enumerate(sample_outputs):
        sample_output = sample_output[len(input_ids[0]):]
        return tokenizer.decode(sample_output, skip_special_tokens=False)

In [ ]:
prompt = '<start>Hello.<start_chat>'
while True:
    prompt += '<Human>'
    prompt += input('Input: ') + '<AI>'
    generated = generate(prompt)
    if '<Human>' in generated:
        generated = generated[: generated.find('<Human>')]
    print(f'AI: {generated}')
    prompt += generated

Input: hello there


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50258 for open-end generation.


AI: good morning. how are you?
Input: I'm doing great. what about you ?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50258 for open-end generation.


AI: i just got off work at the mill. its raining here.


KeyboardInterrupt: Interrupted by user

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Source directory
source_dir = '/content/chats_output1'

# Destination directory in Google Drive
destination_dir = '/content/drive/My Drive/chats_output1'

# Copy the directory and its contents
shutil.copytree(source_dir, destination_dir)

'/content/drive/My Drive/chats_output1'

In [10]:
from transformers import GPT2LMHeadModel

# Path to your pre-trained model in Google Drive
model_path = "/content/drive/MyDrive/chats_output"

# Load the model from your Google Drive
model = GPT2LMHeadModel.from_pretrained(model_path)


In [ ]:
import re
from collections import Counter

# Define the file path
file_path = "test_chats.txt"

# Initialize an empty list to store the parsed conversations
parsed_conversations = []

# Open the file and read its contents with UTF-8 encoding
with open(file_path, "r", encoding="utf-8") as file:
    file_content = file.read()

# Use regular expression to extract each conversation snippet
conversation_snippets = re.findall(r"<start>(.*?)<end>", file_content, re.DOTALL)

# Add each conversation snippet to the parsed_conversations list
parsed_conversations.extend(conversation_snippets)

# Initialize dictionaries to store utterances by speaker and number of utterances by speaker
utterances_by_speaker = {'<human>': '', '<ai>': ''}
num_utterances_by_speaker = {'<human>': 0, '<ai>': 0}

# Initialize variables to store total values for each metric and the number of conversations
total_repetition_rate = 0
total_vocab_size = 0
total_user_rating = 0
total_contextual_relevance = 0
num_conversations = len(parsed_conversations)

def calculate_contextual_relevance(conversation):
    # Define keywords or phrases that indicate the context of the conversation
    context_keywords = ["greeting", "problem", "solution", "thank you", "farewell"]

    # Convert the conversation to lowercase to make keyword matching case-insensitive
    conversation_lower = conversation.lower()

    # Count the number of occurrences of context keywords in the conversation
    total_keywords = sum(conversation_lower.count(keyword) for keyword in context_keywords)

    # Calculate the total number of words in the conversation
    # Here, we split the conversation string into words using whitespace as the delimiter
    total_words = len(conversation_lower.split())

    # Calculate the contextual relevance as the ratio of context keywords to the total number of words
    # You can adjust this formula based on your specific requirements
    if total_words == 0:
        return 0  # Avoid division by zero
    contextual_relevance = total_keywords / total_words

    return contextual_relevance


# Define function to calculate repetition rate
def calculate_repetition_rate(conversation):
    # Split conversation into individual utterances
    utterances = re.findall(r"<(.*?)>", conversation)

    # Count the occurrences of each utterance
    utterance_counts = Counter(utterances)

    # Calculate repetition rate as the proportion of repeated utterances
    repetition_rate = sum(count > 1 for count in utterance_counts.values()) / len(utterances)

    return repetition_rate

# Define function to calculate vocabulary size
def calculate_vocab_size(conversation):
    # Split conversation into words
    words = re.findall(r'\b\w+\b', conversation.lower())

    # Calculate vocabulary size as the number of unique words
    vocab_size = len(set(words))

    return vocab_size

# Iterate over each conversation
for conversation in parsed_conversations:
    # Split the conversation into individual turns
    turns = conversation.strip().split("<start_chat>")
    # Iterate over each turn in the conversation
    for turn in turns:
        # Check if the ">" character exists in the turn
        if ">" in turn:
            # Split the turn into speaker and utterance
            speaker, utterance = turn.split(">", 1)
            # Remove leading and trailing whitespace from speaker and utterance
            speaker = speaker.strip().lower()  # Convert to lowercase
            speaker = '<human>' if speaker == 'human' else '<ai>'
            utterance = utterance.strip()
            # Update the number of utterances for the speaker
            num_utterances_by_speaker[speaker] += 1
            # Update the utterances for the speaker (concatenate the utterance)
            utterances_by_speaker[speaker] += ' ' + utterance

    # Calculate metrics for the current conversation
    repetition_rate = calculate_repetition_rate(conversation)
    vocab_size = calculate_vocab_size(conversation)

    contextual_relevance = calculate_contextual_relevance(conversation)

    # Update total values for each metric
    total_repetition_rate += repetition_rate
    total_vocab_size += vocab_size

    total_contextual_relevance += contextual_relevance

# Calculate average values for each metric
avg_repetition_rate = total_repetition_rate / num_conversations
avg_vocab_size = total_vocab_size / num_conversations

avg_contextual_relevance = total_contextual_relevance / num_conversations

# Format the output into a single string
output = (
    f"Average Repetition Rate: {avg_repetition_rate}\n"
    f"Average Vocabulary Size: {avg_vocab_size}\n"
    f"Average Contextual Relevance: {avg_contextual_relevance}\n"
)

# Print the single output
print(output)


Average Repetition Rate: 0.12873763736263705
Average Vocabulary Size: 105.203
Average Contextual Relevance: 0.0006629980749520235



In [ ]:
import re
import math

# Define the file path
file_path = "test_chats.txt"

# Initialize variables to store total bits and total words
total_bits = 0
total_words = 0

# Open the file and read its contents with UTF-8 encoding
with open(file_path, "r", encoding="utf-8") as file:
    file_content = file.read()

# Tokenize the text into words
words = re.findall(r'\b\w+\b', file_content)

# Calculate the total number of bits needed to represent all words
for word in words:
    total_bits += len(word) * 8  # Assuming each character requires 8 bits

# Calculate the total number of words
total_words = len(words)

# Calculate bits per word
bits_per_word = total_bits / total_words

# Print the result
print("Bits per Word:", bits_per_word)


Bits per Word: 29.790972414313377


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from nltk.translate.bleu_score import corpus_bleu
import nltk
import torch

nltk.download('punkt')

# Move the model to the CUDA device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Add a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Evaluation texts (context passages)
evaluation_contexts = [
    "The quick brown fox jumps over the lazy dog.",
    "A stitch in time saves nine.",
    "Actions speak louder than words."
]

# Questions
questions = [
    "What jumps over the lazy dog?",
    "What saves nine?",
    "What speaks louder than words?"
]

# Reference answers
reference_answers = [
    "The quick brown fox.",
    "A stitch in time.",
    "Actions."
]

evaluation_texts = [(context, question) for context, question in zip(evaluation_contexts, questions)]
references = [[answer] for answer in reference_answers]

# Tokenize and move to device
tokenized_texts = [
    tokenizer(context, question, return_tensors="pt", padding=True, truncation=True).to(device)
    for context, question in evaluation_texts
]
generated_answers = [
    model.generate(
        input_ids=tokenized_text['input_ids'],
        max_length=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    ).to(device)
    for tokenized_text in tokenized_texts
]
generated_answers = [tokenizer.decode(generated_answer[0], skip_special_tokens=True) for generated_answer in generated_answers]
bleu_score = corpus_bleu(references, generated_answers)

print("BLEU score:", bleu_score)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BLEU score: 0.07090387830475452


In [12]:
import collections
import torch

# Function to tokenize text
def get_tokens(text):
    return tokenizer.tokenize(text)

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# Your provided data
evaluation_contexts = [
    "The quick brown fox jumps over the lazy dog.",
    "A stitch in time saves nine.",
    "Actions speak louder than words."
]
questions = [
    "What jumps over the lazy dog?",
    "What saves nine?",
    "What speaks louder than words?"
]
reference_answers = [
    "The quick brown fox.",
    "A stitch in time.",
    "Actions."
]

# Tokenize evaluation texts
evaluation_texts = [(context, question) for context, question in zip(evaluation_contexts, questions)]
references = [[answer] for answer in reference_answers]

# Generate answers using the model
generated_answers = []
for text in evaluation_texts:
    input_ids = tokenizer.encode(text[0], text[1], return_tensors="pt", padding=True, truncation=True)
    generated_answer = model.generate(input_ids=input_ids, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    generated_answers.append(tokenizer.decode(generated_answer[0], skip_special_tokens=True))

# Compute F1 score for each pair of gold and predicted answers
for gold_answer, predicted_answer in zip(reference_answers, generated_answers):
    f1_score = compute_f1(gold_answer, predicted_answer)
    print("F1 Score:", f1_score)


F1 Score: 0.18867924528301885
F1 Score: 0.1851851851851852
F1 Score: 0.11764705882352941
